# pyFEMM Electrostatic Analysis: 

Electrostatic Analysis is used to visualize the voltage potential and electrical field intensity.

# Setup on Windows Operating System

In [ ]:
# Github: Clone typFEMM repo
!pip install tpyfemm@git+https://github.com/12Tall/tpyFEMM

In [ ]:
# If not already installed, Install FEMM and then install pyfemm with the following:
!pip install pyfemm

In [3]:
# Load Libraries
import time # , re

import numpy as np
from scipy.integrate import trapz

import keyboard
import win32gui #, win32con

import femm

import matplotlib.pyplot as plt
plt.style.use('ggplot'); # 'seaborn-white'
from PIL import Image

ModuleNotFoundError: No module named 'keyboard'

In [ ]:
# Start up and connect to FEMM
femm.openfemm(); 

time.sleep(1)

# Move Window
femm_window = win32gui.GetForegroundWindow();
win32gui.MoveWindow(femm_window, 1000, 200, 800, 800, True)

# Create New Electrostatics Document
femm.newdocument(1);

# Change active Window
time.sleep(0.1)
keyboard.press('alt+tab')
keyboard.release('tab')
time.sleep(0.1)
keyboard.release('alt')

# Electrostatics Pre Processing - 'ei'

## Materials

In [ ]:
## Use FEMM library Materials
femm.ei_addmaterial('air',1,1,0);
femm.ei_addmaterial('dielectric',4,4,0);

## Geometry

In [4]:
def addRectGeo(xc, yc, x, y, material, angle=0, mag_angle=0, group=0):  
    # By default North is up, angle rotates counter-clockwise [degrees].  
    x1, y1, x2, y2 = xc-x/2, yc-y/2, xc+x/2, yc+y/2  

    if angle == 0:  
        # Draw
        femm.mi_drawrectangle(x1, y1, x2, y2)  # (x1, y1, y2, x2)  
        femm.mi_addblocklabel(xc, yc)
        femm.mi_selectlabel(xc, yc)
        femm.mi_setblockprop(material, 0, 1, '<None>', mag_angle, group, 0)
        # Create Group
        femm.mi_selectlabel(xc, yc)
        femm.mi_selectrectangle(x1, y1, x2, y2, 4)
        femm.mi_setgroup(group)  # Create Group
    else:
        # Draw
        femm.mi_drawrectangle(x1, y1-100, x2, y2-100) # draw in x,-y quadrant
        femm.mi_addblocklabel(xc, yc-100)
        femm.mi_selectlabel(xc, yc-100)
        femm.mi_setblockprop(material, 0, 1, '<None>', mag_angle, group, 0)
        # Create Group
        femm.mi_selectlabel(xc, yc-100)
        femm.mi_selectrectangle(x1, y1-100, x2, y2-100, 4)  
        femm.mi_setgroup(group)  # Create Group  
        # Move to Final Position
        femm.mi_selectgroup(group)  
        femm.mi_moverotate(xc, yc-100, angle)  
        femm.mi_selectgroup(group)  
        femm.mi_movetranslate(0, 100)  # Return to +x,+y quadrant  

    femm.mi_clearselected()

In [ ]:
# Define Geometry
x_offset = 30
y_offset = 2

# Draw Rectangle Magnets
addRectGeo(25+x_offset, 20+y_offset, 5, 20,
           'NdFeB 32 MGOe', mag_angle=180, group=2)
addRectGeo(33+x_offset, 18+y_offset, 10, 5,
           'NdFeB 32 MGOe', mag_angle=-90, group=3)  # Added 5 offset for howard Johnson Gate

addRectGeo(0, 7, 40, 2,
           'Mumetal', mag_angle=0, group=1)  # Sled Shielding
# Slanted Magnets
addRectGeo(43+x_offset, 20+y_offset, 5, 20, 'NdFeB 32 MGOe',
           -135, mag_angle=0, group=4)
addRectGeo(51+x_offset, 20+y_offset, 5, 20, 'NdFeB 32 MGOe',
           -135, mag_angle=0, group=5)
addRectGeo(59+x_offset, 20+y_offset, 5, 20, 'NdFeB 32 MGOe',
           -135, mag_angle=0, group=6)
addRectGeo(67+x_offset, 20+y_offset, 5, 20, 'NdFeB 32 MGOe',
           -135, mag_angle=0, group=7)
# addRectGeo(75+x_offset, 20+y_offset, 5, 20, 'NdFeB 32 MGOe',
#              -135, mag_angle=0, group=8)

# Mirror Geometry
for i in [1, 2, 3, 4, 5, 6, 7, 8]:  # range(2,3+1)
    femm.ei_selectgroup(i)
    # femm.mi_selectrectangle(-100, -100, 160, 100, 4) # x1,y1,x2,y2,editmode
    femm.ei_mirror(-60, 0, 160, 0)  # x1,y1,x2,y2

# addRectGeo(-22, 0, 2, 16,
#             'Mumetal', mag_angle=0, group=1) # Sled Shielding End Cap
# Sled / Rotor Magnet
addBarMagnet(0, 0, 40, 10, 'NdFeB 32 MGOe',
             mag_angle=180, group=1)  # (x, y, w, h) : 2,10,20,10 works

In [ ]:
## View
femm.ei_zoomnatural()  
femm.ei_zoomout() 
# femm.mi_refreshview()

# Labels

In [ ]:
# Label the boundary material  
femm.ei_addblocklabel(10, 30); # Air
femm.ei_selectlabel(10, 30);
femm.ei_setblockprop('Air', 0, 1, '<None>', 0, 0, 4);
femm.ei_clearselected()

In [5]:
# Define an "open" boundary condition using the built-in function:  
# femm.mi_makeABC()
femm.mi_makeABC(6, 120, 80, 0, 0) # (n, R, x, y, bc), originally '()'

# Set the boundary conditions
femm.mi_addboundprop('outside', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

NameError: name 'femm' is not defined

In [ ]:
## View
femm.mi_zoomnatural()  
# femm.mi_refreshview()

In [ ]:
# We have to give the geometry a name before we can analyze it.
femm.mi_saveas('electrostatic_model.fee');

# Now,analyze the problem and load the solution when the analysis is finished
femm.mi_analyze()
femm.mi_loadsolution()

time.sleep(1)

In [ ]:
# Now, the finished input geometry can be displayed.
femm.mi_zoomnatural()  
# femm.mi_zoom(0, 0, 80, 60)  # x1, y1, x2, y2

In [ ]:
# Save Analysis
femm.mi_savebitmap('electrostatic_output.bmp') # ("filename")  
# femm.mi_savemetafile('Output_Meta')  

In [ ]:
# Display Image
Image.open('Accelerator_Pre_Processing.bmp')

# Magnetics Post Processing - 'mo'

In [ ]:
# Post-Process(legend, gscale, upper_B, lower_B, type); pyFEMM Manual pg.21
femm.mo_showdensityplot(1, 0, 0.000197118052262532, 0.516967928277248, 'bmag')

# Zoom Post Processing
# femm.mo_zoomnatural()
# femm.mo_zoom(0, 0, 80, 60)  # x1, y1, x2, y2

In [ ]:
# Save Image of Port Processing  
femm.mo_savebitmap('Accelerator_Post_Processing.bmp')  

Image.open('Accelerator_Post_Processing.bmp')  

# Analysis

# Shutdown

In [6]:
# When the analysis is completed, FEMM can be shut down.
femm.closefemm();

NameError: name 'femm' is not defined